In [12]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os

In [13]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
llm=ChatGroq(model='llama-3.1-8b-instant')

In [14]:
from langchain_classic.schema import(
    AIMessage,
    HumanMessage,SystemMessage
)

In [15]:
speech='''I have a dream that one day down in Alabama, with its vicious racists, with its governor having his lips dripping with the words of interposition and nullification – one day right there in Alabama little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.

I have a dream today.

I have a dream that one day every valley shall be exalted, and every hill and mountain shall be made low, the rough places will be made plain, and the crooked places will be made straight, and the glory of the Lord shall be revealed and all flesh shall see it together.

This is our hope. This is the faith that I go back to the South with. With this faith we will be able to hew out of the mountain of despair a stone of hope. With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood. With this faith we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day.

This will be the day, this will be the day when all of God’s children will be able to sing with new meaning “My country ’tis of thee, sweet land of liberty, of thee I sing. Land where my father’s died, land of the Pilgrim’s pride, from every mountainside, let freedom ring!'''

In [16]:
chat_message=[
    SystemMessage(content='you are an assistant specialized in summarizing speech'),
    HumanMessage(content=f'please provide a short and conscise summary of the speech. \n\n text:{speech}')
]

In [17]:
llm.get_num_tokens(speech)

314

In [18]:
llm.invoke(chat_message)

AIMessage(content='In this iconic speech, Martin Luther King Jr. expresses his dream of a future where racial equality and harmony exist. He envisions a day when black and white children can hold hands as equals, and where the nation\'s valleys are elevated and its mountains are made low. With faith and determination, King believes Americans can overcome their differences and work together towards freedom, singing "My Country, \'Tis of Thee" with a newfound appreciation for the land of liberty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 361, 'total_tokens': 456, 'completion_time': 0.16697008, 'prompt_time': 0.020184603, 'queue_time': 0.054430737, 'total_time': 0.187154683}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--b415873a-bb6a-47e2-967e-826daea16fd4-0', usage_metadata={'input_tokens'

## Prompt template text summarization

In [19]:
from langchain_classic.chains.llm import LLMChain
from langchain_classic.prompts import PromptTemplate

In [20]:
generictemplate='''
    write a summary of the following speech:
    speech:{speech}
    translate the precise summary in only {language}
'''

In [21]:
prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generictemplate
)
prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\n    write a summary of the following speech:\n    speech:{speech}\n    translate the precise summary in only {language}\n')

In [22]:
complete_prompt=prompt.format(speech=speech,language='French')

In [23]:
llm_chain=LLMChain(prompt=prompt,llm=llm)
summary=llm_chain.run({'speech':speech,'language':'French'})
summary

C:\Users\Chetan\AppData\Local\Temp\ipykernel_11488\1914581972.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(prompt=prompt,llm=llm)
C:\Users\Chetan\AppData\Local\Temp\ipykernel_11488\1914581972.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  summary=llm_chain.run({'speech':speech,'language':'French'})


'Résumé de l\'allocution :\n\nMartin Luther King Jr. exprime son rêve d\'un avenir où les enfants noirs et blancs vivent ensemble en harmonie. Il espère que les régions basses seront exaltées, les collines et les montagnes seront abaissées, et que la gloire de Dieu sera révélée pour que tous les êtres humains la voient ensemble. Il prône la foi qui lui permettra de créer un espoir dans les décisions désespérantes de la nation et de transformer les discords en une symphonie de fraternité.\n\nRésumé en français précis :\n\nMartin Luther King Jr. évoque un avenir idéal où les enfants noirs et blancs sont unis, sans racisme. Il aspire à une époque où toutes les régions seront exaltées, où les collines et les montagnes seront nivelées, et où la gloire de Dieu sera partagée par tous les êtres humains. Il réaffirme sa foi qui lui permet d\'espérer un avenir meilleur et de transformer les conflits en une harmonie fraternelle.\n\nTraduction en français :\n\n"J\'ai un rêve pour l\'avenir. Un jou

## Stuff document chain

In [24]:
from langchain_classic.chains.summarize import load_summarize_chain

In [25]:
from langchain_community.document_loaders import YoutubeLoader

In [40]:
url='https://www.youtube.com/watch?v=tzrwxLNHtRY'
url=url.split('=')[1]
doc=YoutubeLoader(url).load()

In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
doc=RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=50).split_documents(doc)

In [29]:
llm=ChatGroq(model='llama-3.1-8b-instant')

In [35]:
chain=load_summarize_chain(llm,chain_type='map_reduce')

In [37]:
res=chain.run(doc)

In [39]:
llm.get_num_tokens(res)

251

In [49]:
llm.get_num_tokens(doc[0].page_content)

2325